<a href="https://colab.research.google.com/github/tofighi/AnyLogic/blob/main/iris2onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install tf2onnx -q
!pip install onnx -q

In [29]:
import tensorflow
import tf2onnx
print(tensorflow.__version__)
print (tf2onnx.__version__)

2.12.0
1.14.0


In [30]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from tf2onnx.keras2onnx_api import convert_keras
import onnx
import numpy as np

def load_data():
    """ Read and transform the training, validation and test data """
    iris = load_iris()
    X = iris['data']
    y = iris['target']
    names = iris['target_names']
    feature_names = iris['feature_names']

    # One hot encoding
    enc = OneHotEncoder()
    Y = enc.fit_transform(y[:, np.newaxis]).toarray()

    # Scale data to have mean 0 and variance 1
    # which is importance for convergence of the neural network
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the data set into training + val and test
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=1)
    # Split val out of "training + val"
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25*0.8=0.2
    print("Train Shape: ", X_train.shape)
    print("Validation Shape: ", X_val.shape)
    print("Test Shape: ", X_test.shape)
    return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test)


def create_custom_model(input_dim, output_dim, nodes, n=1, name='model'):
    """ Build a Sequential model with the specified sepcs """
    # Create model
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def train_pipeline(layers, nodes_per_layer):
    """ Run thrugh the pipeline of data loading and model building/training """
    trains, vals, tests = load_data()

    n_features = trains[0].shape[1] # x train
    n_classes = trains[1].shape[1] # y train
    print ("# features: ", n_features)
    print ("# classes: ", n_classes)

    model = create_custom_model(n_features, n_classes, nodes_per_layer, layers, 'model_1')

    print(model.summary())

    cb = EarlyStopping(monitor='loss', patience=10, min_delta=0.005)


    verbose = False  # or False, depending on your preference
    _ = model.fit(*trains,
              batch_size=64,
              epochs=200,
              validation_data=vals,
              callbacks=[cb],
              verbose=1 if verbose else 0
              )

    score = model.evaluate(*tests, verbose=0)
    return model, score


def save_to_onnx(model, filename):
    """ Save a keras model as an ONNX file """
    proto = convert_keras(model)

    if not filename.endswith(".onnx"): filename += ".onnx"
    onnx.save(proto, filename)

#Run and Save Model
model, score = train_pipeline(1, 8)
print("Score:", score)

save_to_onnx(model, "iris.onnx")



Train Shape:  (90, 4)
Validation Shape:  (30, 4)
Test Shape:  (30, 4)
# features:  4
# classes:  3
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 8)                 40        
                                                                 
 dense_45 (Dense)            (None, 3)                 27        
                                                                 
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________
None
Score: [0.5029127597808838, 0.7333333492279053]
